In [ ]:
# Imports
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
# Set up the executable path needed to scrape the webpage
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# Set up a dictionary to store the info to be returned
mars = {}

## Find the first article title and summary paragraph

In [ ]:
# Open the website to find news about mars
news_url = "https://redplanetscience.com/"
browser.visit(news_url)

# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Retrieve the first element with the news title, and the first element with the paragraph text
news_title = soup.find('div', class_='content_title').text
news_p = soup.find('div', class_='article_teaser_body').text

# Add the title and paragraph to the mars dictionary
mars['news_title'] = news_title
mars['news_paragraph'] = news_p

## Find the featured image

In [ ]:
# Open the website to scrape and find the featured space image
images_url = 'https://spaceimages-mars.com/'
browser.visit(images_url)

In [ ]:
# HTML object
html2 = browser.html

# Parse HTML with Beautiful Soup
soup2 = BeautifulSoup(html2, 'html.parser')

# Retrieve the image url
image = soup2.find('img', class_='headerimage fade-in')
link_to_image = image['src']
featured_image_url = images_url + link_to_image

# Add the image url to the mars_data dictionary
mars['featured_image_url'] = featured_image_url

## Find some fun facts about Mars!

In [ ]:
# Open the Mars facts website
facts_url = 'https://galaxyfacts-mars.com/'
browser.visit(facts_url)

In [ ]:
# Scrape the website to grab the table with Mars and Earth information
tables = pd.read_html(facts_url)
facts = tables[0]

In [ ]:
# Set the first row in the table to be the column names
facts.columns = facts.iloc[0]

In [ ]:
# Drop the row of data used to name the table columns
facts = facts.drop(facts.index[0])

In [ ]:
# Reset the index of the table
facts = facts.set_index('Mars - Earth Comparison')

In [ ]:
# Write the dataframe into an HTML script
html_table = facts.to_html()

In [ ]:
# Clean up the html code
html_table = html_table.replace('\n', '')

In [ ]:
# Add the html table to the mars_data dictionary
mars['facts_table'] = html_table

## Scrape the hemisphere images

In [ ]:
# Open the website that has the photos of Mars' hemispheres
hemispheres_url = "https://marshemispheres.com/"
browser.visit(hemispheres_url)

# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# Find find links to each hemisphere page
thumbnails = soup.find_all('div', class_='description')

In [ ]:
# Create a list to store dictionaries holding the hemisphere image info
hemisphere_img_urls = []


# Loop through each thumbnail element 
for thumb in thumbnails:
    
    # Store the title of the image
    title = thumb.find('h3').text
    
    # Find the html link to the high res image page
    link = thumb.find('a')['href']

    # Open the page that has the high res image
    image_page_url = hemispheres_url + link
    browser.visit(image_page_url)

    # Create a new HTML object
    image_page_html = browser.html

    # Parse HTML with Beautiful Soup
    image_soup = BeautifulSoup(image_page_html, 'html.parser')
    
    # Find the image tag, pull out the src, append the image link to the original url
    image_link = image_soup.find('img', class_='wide-image')['src']
    img_url = hemispheres_url + image_link
    
    # Add the title and image url as a dictionary to the hemisphere image urls list
    hemisphere_img_urls.append({'title': title, 'img_url': img_url})

In [ ]:
# Add the hemisphere image urls to the mars_data dictionary
mars['hemisphere_img_urls'] = hemisphere_img_urls

In [ ]:
# Quit the browser
browser.quit()